<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Buffer Pool Analysis

_by Grayson Badgley, September 17, 2020_

This notebook performs analysis of ...

## Methodology

...

## References

...

- Data:
  https://ww3.arb.ca.gov/cc/capandtrade/offsets/issuance/arboc_issuance.xlsx


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import intake

# open our data catalog
cat = intake.open_catalog("./catalog.yaml")

### Load data


In [ ]:
df = cat.arboc_issuance.read()

display(df.head())

## Cleaning the data

Looks like two of the the Forest projects are reforestation and have "Forest
Buffer Contributions" of "reforest defer" -- they haven't been issued any
credits so lets exclude them from the rest of the analysis.

It also looks like one project has received zero credits -- let's just remove
that to be conservative.

Then we cut things down to just forests, because we want to understand those
projects and their relationship to the _forest_ buffer pool.


In [ ]:
df[df["Forest Buffer Account Contribution"].apply(lambda x: isinstance(x, str))]

In [ ]:
df = df[
    ~df["Forest Buffer Account Contribution"].apply(
        lambda x: isinstance(x, str)
    )
]
df = df[df["ARB Offset Credits Issued"] > 0]
forest_df = df[df["Project Type"] == "Forest"]

Both EA and COP contriubte to buffer pool, we'll consider both


In [ ]:
forest_df.groupby("Early Action/ Compliance")[
    "Forest Buffer Account Contribution"
].sum()

There are 127 total forest projects that have ever been issued credits


In [ ]:
n_projects = forest_df["OPR Project ID"].nunique()
n_projects

## ACR260/CAFR5214

Has been issued credits three times, though it has applied for them four times.
The ARB excel file notes that the reporting period starting 2016-09-27 was I
confirmed that the reporting period start and end date match the Annual OPDRs
from the ACR web portal:
https://acr2.apx.com/mymodule/reg/TabDocuments.asp?r=111&ad=Prpt&act=update&type=PRO&aProj=pub&tablename=doc&id1=260.


In [ ]:
proj_id = "ACR260"
forest_df[forest_df["OPR Project ID"] == proj_id]

## How big is this project?

In the scheme of things, this is a pretty big project -- it's the 15th of 128
projects in the CARB issuance database. It's also the largest in Oregon.


In [ ]:
total_credits = forest_df["ARB Offset Credits Issued"].sum()
per_proj_credits = forest_df.groupby("OPR Project ID")[
    "ARB Offset Credits Issued"
].sum()
perc_credits = per_proj_credits / total_credits * 100

proj_state_map = (
    forest_df.groupby("OPR Project ID").State.max().to_dict()
)  # max() makes 1:1 mapping of opr_id to state
perc_credits = perc_credits.sort_values(ascending=False).to_frame()
perc_credits["state"] = perc_credits.index.map(proj_state_map)
perc_credits = perc_credits.rename(
    columns={
        "ARB Offset Credits Issued": "Percent Forest ARB Offset Credits Issued"
    }
)

proj_loc = perc_credits.index.get_loc(proj_id)
perc_credits[: proj_loc + 1]  # +1 to include ACR16, otherwise off by one

In [ ]:
total_buffer_pool = forest_df["Forest Buffer Account Contribution"].sum()

In [ ]:
total_buffer_pool

In [ ]:
proj_credits = forest_df[forest_df["OPR Project ID"] == proj_id][
    "ARB Offset Credits Issued"
].sum()
proj_credits

In [ ]:
proj_credits / total_buffer_pool

In [ ]:
frac_buffer = 0.107
frac_burned = 0.8
prob_mortality = 0.25
freq_burn = 0.50
arr = (
    np.random.binomial(size=(1000, 100), n=1, p=freq_burn)
    * frac_buffer
    * frac_burned
    * prob_mortality
)
# ser[ser > 1] = 1

In [ ]:
g = pd.DataFrame(arr.cumsum(axis=1)).T.plot(
    color="grey", alpha=0.2, legend=False
)
med_case = np.median(arr.cumsum(axis=1), axis=0)
g.plot(np.arange(100), med_case, lw=5, c="k")
sns.despine()

In [ ]:
forest_df["buffer_contrib"] = (
    forest_df["Forest Buffer Account Contribution"]
    / forest_df["ARB Offset Credits Issued"]
)

In [ ]:
# for understanding fire contribution, lets get rid of ea, since have some funky high values
cop_forest = forest_df[forest_df["Early Action/ Compliance"] == "COP"]

In [ ]:
(
    ((0.5 * 2 + 0.5 * 4) / 100) / cop_forest["buffer_contrib"].astype(float)
).median()

In [ ]:
(0.04 / cop_forest["buffer_contrib"].astype(float)).median()